Subsumption is a terminology for a redundant fact

Forward subsumption  is deleting new redundant inferences. We need to compare a new fact and see if it is made redundant by our large database of old facts

Backward subsumption is the opposite. We want to see which pieces of our old big datbase are made redunant by our new fact

If I knew x > 0 and then derive x > -1, this new fact is redundant.
If I knew x > 0 and derive x > 1, then the old fact x > 0 is made redundant. The new fact is strictly stronger.

Some common notions of subsumption
- conditional facts with weaker assumtions. `A /\ B => C` is strictly weaker than `A => C` 
- intervals
- Unification patterns. `forall A, f(A)= g(A)` is stronger than `f(1) = g(1)`. Likewise `forall A B, g(A,B) = 7` is stronger than `forall A, g(A,A) = 7` 
- Linear equations
- Linear Inequalities
- Polynomial equations
- Boolean formulas. 

Forward subsumption of linear inequalities is answerable via linear programming. Given the database `Ax >= b`, is `cx >= d` redundant? Well the answer to `min cx s.t. Ax >= b` will tell you. If the min was already greater than d, it is redundant, and the dual vector tells you how to derive this fact

Forward Subsumption of boolean variables can be asked via a SAT query. We can have a big database and push and pop small queries on.

It is less clear to me how to do backward subsumption.

We can normalize


Grobner bases and guassian elimination
Duplicating inside z3.

$Ax=b$ This represents some affine subspace
Does this hyperplane
$cx=d$
lie inside the subspace?

$l(Ax - b) = cx - d$

The dual problem
$ lb=d $
$ lA=c $

So we could solve this via a stack matrix $l(A|b) = (c|d)$
Or we could do a block LU.

What are my options for decompositions. LU, QR, eigen, svd


https://github.com/vprover/vampire/pull/546 vampire pull request about using SAT for subsumption. Paper references
- 2022: "First-Order Subsumption via SAT Solving." by Jakob Rath, Armin Biere and Laura Kovács
- 2023: "SAT-Based Subsumption Resolution" by Robin Coutelier, Jakob Rath, Michael Rawson and Laura Kovács
- 2024: "SAT Solving for Variants of First-Order Subsumption" by Robin Coutelier, Jakob Rath, Michael Rawson, Armin Biere and Laura Kovács





Polynomials are just another container.
bottom up ematching still seems ok
polynomials are ground from the term reqwriting perspective.
grobner guaranteed to terminate?
Also linear equalities

We can also do SMT guarded patterns.



linear inequalities?

Knuth bendix and grobner
https://leodemoura.github.io/files/synasc.pdf passmore de moura Superfluous S-polynomials in Strategy-Independent
Gr¨obner Bases
https://www.sciencedirect.com/science/article/pii/S0747717199902626  A Categorical Critical-pair Completion Algorithm
https://dl.acm.org/doi/pdf/10.1145/74540.74548  Knuth-Bendix procedure and Buchberger algorithm: a synthesis
https://link.springer.com/chapter/10.1007/978-3-0348-8800-4_9 Normalized Rewriting: A Unified View of Knuth-Bendix Completion and Gröbner Bases Computation
https://arxiv.org/abs/math/9812097  Applications of Rewriting Systems and Groebner Bases to Computing Kan Extensions and Identities Among Relations


https://jingnanshi.com/blog/groebner_basis.html
https://colab.research.google.com/github/bernalde/QuIP/blob/master/notebooks/Notebook%202%20-%20Groebner%20basis.ipynb hmm. quantum integer programming?
https://github.com/sumiya11/Groebner.jl?tab=readme-ov-file
msolve
https://www.philipzucker.com/computing-syzygy-modules-in-sympy/


Also polynomial factoring. Normalization of the poly could involve slapping the factors in the egraph.


In [1]:
import sympy as sp

In [ ]:
class UnionFind():
    def __init__(self):
        self.uf = []
    def makeset(self):
        uf = self.uf
        uf.append(len(uf))
        return len(uf) - 1
    def find(self,x):
        while self.uf[x] != x:
            x = self.uf[x]
        return x
    def union(self,x,y):
        x = self.find(x)
        y = self.find(y)
        self.uf[x] = y
        return y
    def __len__(self):
        return len(self.uf)
    def __repr__(self):
        return "UnionFind({})".format(self.uf)

EClass = int

from dataclasses import dataclass
from typing import Any, Dict

@dataclass(frozen=True)
class Lit():
    data: Any
    def rebuild(self):
        return self

@dataclass(frozen=True)
class ENode():
    head: Any
    args : tuple[EClass]
    def rebuild(self):
        return ENode(self.head, tuple(map(egraph.uf.find, self.args)))

In [24]:
# contextual simplification
from z3 import *
#ArraySort(IntSort(), IntSort())
x, y, z = Ints("x y z")
a = Array("a", IntSort(), IntSort())
z3.simplify(y / (y * x))
z3.simplify(Select(Store(a, y, IntVal(0)), x))
#z3.simplify(Implies(x == y, ))
e = Select(Store(a, y, IntVal(0)), x)
z3.simplify(e)
z3.simplify(Implies(x == y, z == e)) # nope Ok, well it was an idea

# z3 mirroring. Do quantifier instan outside z3. Is this of any relation to avatar?



Or(Not(x == y), z == Store(a, y, 0)[x])